# 데이터 전처리/시각화 3 GroupBy · 결합 · 시간/문자열 · apply/map 마스터 (요약테이블 2개 만들기)

In [5]:
import pandas as pd
import numpy as np

In [6]:
# 문제 상황(가격 문자열, 결측, 결제 값 혼합, 메뉴 표기 흔들림)
raw = [
    {"order_id":"O001","date":"2026-01-01 09:12","store":"광교점","menu":"Americano","price":"4,500원","qty":"2","paid":"TRUE","channel":"kiosk"},
    {"order_id":"O002","date":"2026/01/01 10:05","store":"광교점","menu":" Latte ","price":"5000","qty":1,"paid":"True","channel":"app"},
    {"order_id":"O003","date":"2026-01-02 12:20","store":"광교점","menu":"Mocha","price":None,"qty":2,"paid":"FALSE","channel":"kiosk"},
    {"order_id":"O004","date":"2026-01-03 15:40","store":"수원점","menu":"Americano","price":"4500","qty":None,"paid":True,"channel":"app"},
    {"order_id":"O005","date":"2026-01-03 18:10","store":"수원점","menu":"latte","price":"5,000원","qty":"3","paid":"TRUE","channel":"kiosk"},
    {"order_id":"O006","date":"2026-01-04 08:55","store":"수원점","menu":"Vanilla Latte","price":"5800원","qty":"1","paid":"TRUE","channel":"app"},
    {"order_id":"O007","date":"2026-01-04 09:10","store":"광교점","menu":"Mocha","price":"5500","qty":"1","paid":"FALSE","channel":"kiosk"},
    {"order_id":"O008","date":"2026-01-05 11:00","store":"광교점","menu":"Americano","price":"4500원","qty":"1","paid":"TRUE","channel":"app"},
]

df = pd.DataFrame(raw)
df.head()

,order_id,date,store,menu,price,qty,paid,channel
0,O001,2026-01-01 09:12,광교점,Americano,"4,500원",2,TRUE,kiosk
1,O002,2026/01/01 10:05,광교점,Latte,5000,1,True,app
2,O003,2026-01-02 12:20,광교점,Mocha,None,2,FALSE,kiosk
3,O004,2026-01-03 15:40,수원점,Americano,4500,None,True,app
4,O005,2026-01-03 18:10,수원점,latte,"5,000원",3,TRUE,kiosk


In [7]:
# 1차 정제 (문자열/결측/타입 통일)
# (1) 날짜 → datetime 변환 + 파생피처 만들기
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day_name"] = df["date"].dt.day_name()  # 요일(영문)
df["ym"] = df["date"].dt.to_period("M").astype(str)  # "2026-01" 형태

In [8]:
# (2) 메뉴 문자열 정리 (strip + 표기 통일)
df["menu"] = df["menu"].astype(str).str.strip().str.title()

In [9]:
# (3) price 정리 ("4,500원" → 4500)

df["price"] = (
    df["price"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
)
df["price"] = pd.to_numeric(df["price"], errors="coerce")

In [10]:
# (4) qty 정리 (결측 처리 + 숫자형)
df["qty"] = pd.to_numeric(df["qty"], errors="coerce")
df["qty"] = df["qty"].fillna(1)  # 실무에서 1로 대체하는 간단 예시(전략 설명)
df["qty"] = df["qty"].astype(int)

In [11]:
# (5) paid 정리 (TRUE/False/True 혼합 → bool)
def to_bool(x):
    if isinstance(x, bool):
        return x
    x = str(x).strip().lower()
    return x in ["true", "1", "yes", "y"]

df["paid"] = df["paid"].apply(to_bool)

In [12]:
# (6) 매출 컬럼 만들기 (paid가 True일 때만 매출 인정)
df["sales"] = np.where(df["paid"], df["price"] * df["qty"], 0)    #np.where(조건, 참일때값, 거짓일때값)
df[["order_id","date","store","menu","price","qty","paid","sales"]].head()

# ※ 여기까지가 “분석 가능한 테이블 만들기”의 핵심 전처리입니다.

,order_id,date,store,menu,price,qty,paid,sales
0,O001,2026-01-01 09:12:00,광교점,Americano,4500.0,2,True,9000.0
1,O002,NaT,광교점,Latte,5000.0,1,True,5000.0
2,O003,2026-01-02 12:20:00,광교점,Mocha,NaN,2,False,0.0
3,O004,2026-01-03 15:40:00,수원점,Americano,4500.0,1,True,4500.0
4,O005,2026-01-03 18:10:00,수원점,Latte,5000.0,3,True,15000.0


## GroupBy 기초 (단일 기준 집계)

In [13]:
# (1) 매장별 총매출 / 주문수 / 결제성공률

store_summary = df.groupby("store").agg(
    total_sales=("sales","sum"),
    orders=("order_id","count"),
    paid_rate=("paid","mean")
).reset_index()

store_summary

,store,total_sales,orders,paid_rate
0,광교점,18500.0,5,0.6
1,수원점,25300.0,3,1.0


### - df.groupby("store")
    - → 데이터를 **store(매장)별로 묶고**
- .agg(...)
    - → 각 매장 그룹에 대해 **요약값(집계)**을 계산합니다.
    - total_sales = sales의 합(sum) → **총 매출**
    - orders = order_id 개수(count) → **주문 건수**
    - paid_rate = paid의 평균(mean) → **결제 비율**
        - (bool은 True=1, False=0처럼 계산돼서 평균이 “True 비율”이 됩니다)
- .reset_index()
    - → store가 인덱스로 올라가 있는 걸 다시 **일반 컬럼으로 내려서** 보기 좋게 만듭니다.
- paid_rate가 **평균(mean)** 이 되는 이유: True=1, False=0처럼 동작해서 “결제 성공 비율”이 됩니다.

## GroupBy 심화 (다중 기준 + agg + 피벗)

In [14]:
# **(1) 월(ym) × 요일(day_name) 매출/주문/결제율**

ym_day = df.groupby(["ym","day_name"]).agg(
    total_sales=("sales","sum"),
    orders=("order_id","count"),
    paid_rate=("paid","mean")
)
ym_day

total_sales  orders  paid_rate
ym      day_name                                
2026-01 Friday            0.0       1        0.0
        Monday         4500.0       1        1.0
        Saturday      19500.0       2        1.0
        Sunday         5800.0       2        0.5
        Thursday       9000.0       1        1.0

In [15]:
# (2) MultiIndex 결과 다루기: reset_index

ym_day = ym_day.reset_index()
ym_day

,ym,day_name,total_sales,orders,paid_rate
0,2026-01,Friday,0.0,1,0.0
1,2026-01,Monday,4500.0,1,1.0
2,2026-01,Saturday,19500.0,2,1.0
3,2026-01,Sunday,5800.0,2,0.5
4,2026-01,Thursday,9000.0,1,1.0


In [16]:
# (3) 피벗 형태로 바꾸기(리포트용): unstack/pivot
pivot_sales = ym_day.pivot(index="ym", columns="day_name", values="total_sales").fillna(0) 
pivot_sales

# ※ 요약표 A 후보 완성: “월별-요일별 매출 피벗”

day_name,Friday,Monday,Saturday,Sunday,Thursday
ym,,,,,
2026-01,0.0,4500.0,19500.0,5800.0,9000.0


## 데이터 결합 (merge / join / concat 차이 체감)

In [17]:
# (1) 카테고리 매핑 테이블 만들기

menu_map = pd.DataFrame([
    {"menu":"Americano", "category":"Coffee"},
    {"menu":"Latte", "category":"Coffee"},
    {"menu":"Mocha", "category":"Coffee"},
    {"menu":"Vanilla Latte", "category":"Latte Variations"},
])

menu_map

,menu,category
0,Americano,Coffee
1,Latte,Coffee
2,Mocha,Coffee
3,Vanilla Latte,Latte Variations


In [18]:
# (2) merge로 결합 (키 기반 결합)

df2 = df.merge(menu_map, on="menu", how="left")
df2[["menu","category"]].drop_duplicates()

# - how="left": 원본(df)의 행은 유지하면서, 매칭되는 카테고리를 붙임
# - 만약 메뉴가 매핑표에 없으면 category가 NaN으로 남음(현업에서 “매핑 누락” 탐지 포인트)

,menu,category
0,Americano,Coffee
1,Latte,Coffee
2,Mocha,Coffee
5,Vanilla Latte,Latte Variations


## apply / map (언제 쓰는가)

In [19]:
# (1) map(딕셔너리 매핑): 채널 한글화 같은 “치환 작업”

channel_map = {"kiosk":"키오스크", "app":"앱"}
df2["channel_kr"] = df2["channel"].map(channel_map).fillna("기타")
df2[["channel","channel_kr"]].drop_duplicates()

,channel,channel_kr
0,kiosk,키오스크
1,app,앱


In [20]:
# (2) apply: 행 단위 규칙이 필요할 때(벡터화보다 느릴 수 있음)
def sales_grade(row):
    if row["sales"] >= 10000:
        return "A"
    elif row["sales"] > 0:
        return "B"
    else:
        return "C"

df2["grade"] = df2.apply(sales_grade, axis=1)
df2[["order_id","sales","grade"]]


,order_id,sales,grade
0,O001,9000.0,B
1,O002,5000.0,B
2,O003,0.0,C
3,O004,4500.0,B
4,O005,15000.0,A
5,O006,5800.0,B
6,O007,0.0,C
7,O008,4500.0,B


## 종합 실습 (오늘의 목표: 요약 테이블 2개 만들기)

In [21]:
# ※ 요약 테이블 A: “월별 요일 매출 피벗 + 결제율”
summary_A = df.groupby(["ym","day_name"]).agg(
    total_sales=("sales","sum"),
    paid_rate=("paid","mean")
).reset_index()

summary_A_pivot = summary_A.pivot(index="ym", columns="day_name", values="total_sales").fillna(0)

summary_A_pivot

day_name,Friday,Monday,Saturday,Sunday,Thursday
ym,,,,,
2026-01,0.0,4500.0,19500.0,5800.0,9000.0


In [22]:
# ※ 요약 테이블 B: “메뉴별(카테고리 포함) 매출/수량 TOP”
menu_summary = df2.groupby(["category","menu"]).agg(
    total_sales=("sales","sum"),
    total_qty=("qty","sum"),
    orders=("order_id","count"),
    paid_rate=("paid","mean")
).reset_index()

menu_summary = menu_summary.sort_values("total_sales", ascending=False)
menu_summary

menu_summary = menu_summary.sort_values(by=["category", "menu", "total_sales"],
                                        ascending=[True, True, True]).reset_index(drop=True)

menu_summary

,category,menu,total_sales,total_qty,orders,paid_rate
0,Coffee,Americano,18000.0,4,3,1.0
1,Coffee,Latte,20000.0,4,2,1.0
2,Coffee,Mocha,0.0,3,2,0.0
3,Latte Variations,Vanilla Latte,5800.0,1,1,1.0


## 저장 & 재현성(검증 로드까지)


In [23]:
# (1) 저장
import os
os.makedirs("data", exist_ok=True)

summary_A.to_csv("data/summary_A_long.csv", index=False)
summary_A_pivot.to_csv("data/summary_A_pivot.csv")
menu_summary.to_csv("data/summary_B_menu.csv", index=False)

In [24]:
# (2) 다시 로드해서 확인(“내가 저장한 게 맞는지”)

check = pd.read_csv("data/summary_B_menu.csv")
check.head()

,category,menu,total_sales,total_qty,orders,paid_rate
0,Coffee,Americano,18000.0,4,3,1.0
1,Coffee,Latte,20000.0,4,2,1.0
2,Coffee,Mocha,0.0,3,2,0.0
3,Latte Variations,Vanilla Latte,5800.0,1,1,1.0


# 최종 미니 과제(제출형)

## 과제 목표: 리포트용 요약표 2개 제출

- summary_A_long.csv 또는 summary_A_pivot.csv 중 1개
- summary_B_menu.csv (매출 기준 TOP 정렬 포함)

### 제출 체크리스트

- 결측/문자열/날짜 정제 코드 포함
- groupby + agg 사용 흔적
- merge(카테고리 결합) 포함
- 저장 파일 2개 이상 생성

In [49]:
# 문제 상황(가격 문자열, 결측, 결제 값 혼합, 메뉴 표기 흔들림)
import pandas as pd
import numpy as np

raw = [
    {"order_id":"O001","date":"2026-01-01 09:12","store":"광교점","menu":"Americano","price":"4,500원","qty":"2","paid":"TRUE","channel":"kiosk"},
    {"order_id":"O002","date":"2026/01/01 10:05","store":"광교점","menu":" Latte ","price":"5000","qty":1,"paid":"True","channel":"app"},
    {"order_id":"O003","date":"2026-01-02 12:20","store":"광교점","menu":"Mocha","price":None,"qty":2,"paid":"FALSE","channel":"kiosk"},
    {"order_id":"O004","date":"2026-01-03 15:40","store":"수원점","menu":"Americano","price":"4500","qty":None,"paid":True,"channel":"app"},
    {"order_id":"O005","date":"2026-01-03 18:10","store":"수원점","menu":"latte","price":"5,000원","qty":"3","paid":"TRUE","channel":"kiosk"},
    {"order_id":"O006","date":"2026-01-04 08:55","store":"수원점","menu":"Vanilla Latte","price":"5800원","qty":"1","paid":"TRUE","channel":"app"},
    {"order_id":"O007","date":"2026-01-04 09:10","store":"광교점","menu":"Mocha","price":"5500","qty":"1","paid":"FALSE","channel":"kiosk"},
    {"order_id":"O008","date":"2026-01-05 11:00","store":"광교점","menu":"Americano","price":"4500원","qty":"1","paid":"TRUE","channel":"app"},
]

df = pd.DataFrame(raw)
df.head()

,order_id,date,store,menu,price,qty,paid,channel
0,O001,2026-01-01 09:12,광교점,Americano,"4,500원",2,TRUE,kiosk
1,O002,2026/01/01 10:05,광교점,Latte,5000,1,True,app
2,O003,2026-01-02 12:20,광교점,Mocha,None,2,FALSE,kiosk
3,O004,2026-01-03 15:40,수원점,Americano,4500,None,True,app
4,O005,2026-01-03 18:10,수원점,latte,"5,000원",3,TRUE,kiosk


In [50]:
# 문자열 / 숫자 / 날짜 정리

# 메뉴 통일
df["menu"] = df["menu"].str.strip().str.title()

# price 숫자로 변환 및 정리
df["price"] = (
    df["price"]
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
)
df["price"] = pd.to_numeric(df["price"], errors="coerce")

# qty → 숫자
df["qty"] = pd.to_numeric(df["qty"], errors="coerce").fillna(0).astype(int)

# paid → boolean
df["paid"] = (
    df["paid"]
    .astype(str)
    .str.upper()
    .map({"TRUE": True, "FALSE": False})
)

# date → datetime + 파생
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["ym"] = df["date"].dt.to_period("M").astype(str)
df["day_name"] = df["date"].dt.day_name()

df

,order_id,date,store,menu,price,qty,paid,channel,ym,day_name
0,O001,2026-01-01 09:12:00,광교점,Americano,4500.0,2,True,kiosk,2026-01,Thursday
1,O002,NaT,광교점,Latte,5000.0,1,True,app,NaT,NaN
2,O003,2026-01-02 12:20:00,광교점,Mocha,NaN,2,False,kiosk,2026-01,Friday
3,O004,2026-01-03 15:40:00,수원점,Americano,4500.0,0,True,app,2026-01,Saturday
4,O005,2026-01-03 18:10:00,수원점,Latte,5000.0,3,True,kiosk,2026-01,Saturday
5,O006,2026-01-04 08:55:00,수원점,Vanilla Latte,5800.0,1,True,app,2026-01,Sunday
6,O007,2026-01-04 09:10:00,광교점,Mocha,5500.0,1,False,kiosk,2026-01,Sunday
7,O008,2026-01-05 11:00:00,광교점,Americano,4500.0,1,True,app,2026-01,Monday


In [51]:
# (6) 매출 컬럼 만들기 (paid가 True일 때만 매출 인정)
df["sales"] = np.where(df["paid"], df["price"] * df["qty"], 0)    #np.where(조건, 참일때값, 거짓일때값)
df[["order_id","date","store","menu","price","qty","paid","sales"]].head()

,order_id,date,store,menu,price,qty,paid,sales
0,O001,2026-01-01 09:12:00,광교점,Americano,4500.0,2,True,9000.0
1,O002,NaT,광교점,Latte,5000.0,1,True,5000.0
2,O003,2026-01-02 12:20:00,광교점,Mocha,NaN,2,False,0.0
3,O004,2026-01-03 15:40:00,수원점,Americano,4500.0,0,True,0.0
4,O005,2026-01-03 18:10:00,수원점,Latte,5000.0,3,True,15000.0


In [52]:
# 테이블 조인 (merge)

menu_map = pd.DataFrame([
    {"menu": "Americano", "category": "Coffee"},
    {"menu": "Latte", "category": "Coffee"},
    {"menu": "Vanilla Latte", "category": "Coffee"},
    {"menu": "Mocha", "category": "Coffee"},
])

df = df.merge(menu_map, on="menu", how="left")

df

,order_id,date,store,menu,price,qty,paid,channel,ym,day_name,sales,category
0,O001,2026-01-01 09:12:00,광교점,Americano,4500.0,2,True,kiosk,2026-01,Thursday,9000.0,Coffee
1,O002,NaT,광교점,Latte,5000.0,1,True,app,NaT,NaN,5000.0,Coffee
2,O003,2026-01-02 12:20:00,광교점,Mocha,NaN,2,False,kiosk,2026-01,Friday,0.0,Coffee
3,O004,2026-01-03 15:40:00,수원점,Americano,4500.0,0,True,app,2026-01,Saturday,0.0,Coffee
4,O005,2026-01-03 18:10:00,수원점,Latte,5000.0,3,True,kiosk,2026-01,Saturday,15000.0,Coffee
5,O006,2026-01-04 08:55:00,수원점,Vanilla Latte,5800.0,1,True,app,2026-01,Sunday,5800.0,Coffee
6,O007,2026-01-04 09:10:00,광교점,Mocha,5500.0,1,False,kiosk,2026-01,Sunday,0.0,Coffee
7,O008,2026-01-05 11:00:00,광교점,Americano,4500.0,1,True,app,2026-01,Monday,4500.0,Coffee


In [53]:
summary_A = df.groupby(["ym","day_name"]).agg(
    total_sales=("sales","sum"),
    paid_rate=("paid","mean")
).reset_index()

summary_A_pivot = summary_A.pivot(index="ym", columns="day_name", values="total_sales").fillna(0)

summary_A_pivot

day_name,Friday,Monday,Saturday,Sunday,Thursday
ym,,,,,
2026-01,0.0,4500.0,15000.0,5800.0,9000.0


In [ ]:
menu_summary = df.groupby(["category","menu"]).agg(
    total_sales=("sales","sum"),
    total_qty=("qty","sum"),
    orders=("order_id","count"),
    paid_rate=("paid","mean")
).reset_index()

menu_summary = (
    menu_summary
    .sort_values("total_sales", ascending=False)
    .reset_index(drop=True)
)

menu_summary

,category,menu,total_sales,total_qty,orders,paid_rate
0,Coffee,Latte,20000.0,4,2,1.0
1,Coffee,Americano,18000.0,4,3,1.0
2,Latte Variations,Vanilla Latte,5800.0,1,1,1.0
3,Coffee,Mocha,0.0,3,2,0.0


In [59]:
# (1) 저장
import os
os.makedirs("data", exist_ok=True)

summary_A_pivot.to_csv("data/summary_A_pivot_review.csv")
menu_summary.to_csv("data/summary_B_menu_review.csv", index=False)

In [ ]:
check = pd.read_csv("data/summary_A_pivot_review.csv")
check.head()

,ym,Friday,Monday,Saturday,Sunday,Thursday
0,2026-01,0.0,4500.0,19500.0,5800.0,9000.0
